In [23]:
pip install numpy-financial

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
import numpy as np
from datetime import date
import numpy_financial as npf
from collections import OrderedDict
from dateutil.relativedelta import *

In [2]:
interest_rate = .04
years = 30
payments_year = 12
principal = 200000
addl_princ = 50
start_date = (date(2016,1,1))

In [25]:
pmt = npf.pmt(interest_rate/payments_year, years*payments_year, principal)

In [26]:
pmt

-954.8305909309076

In [27]:
# Period to calculate
per = 1

# Calculate the interest
ipmt = npf.ipmt(interest_rate/payments_year, per, years*payments_year, principal)

# calculate the principal
ppmt = npf.ppmt(interest_rate/payments_year, per, years*payments_year, principal)

print(ipmt, ppmt)

-666.6666666666667 -288.1639242642409


In [28]:
per  = 240

ipmt = npf.ipmt(interest_rate/payments_year, per, years*payments_year, principal)

ppmt = npf.ppmt(interest_rate/payments_year, per, years*payments_year, principal)

print(ipmt, ppmt)

-316.49041533656924 -638.3401755943385


In [7]:
# built the table
rng = pd.date_range(start_date, periods=years*payments_year, freq="MS")
rng.name = "Payment_Date"

In [12]:
df = pd.DataFrame(index=rng, columns=['Payment', 'Principal', 'Interest', 'Addl_principal', 'Balance'], dtype='float')
df.reset_index(inplace=True)
df.index +=1
df.index.name = "Period"

In [13]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_principal,Balance
Period,,,,,,
1,2016-01-01,NaN,NaN,NaN,NaN,NaN
2,2016-02-01,NaN,NaN,NaN,NaN,NaN
3,2016-03-01,NaN,NaN,NaN,NaN,NaN
4,2016-04-01,NaN,NaN,NaN,NaN,NaN
5,2016-05-01,NaN,NaN,NaN,NaN,NaN


In [29]:
df["Payment"] = npf.pmt(interest_rate/payments_year, years*payments_year, principal)

In [17]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_principal,Balance
Period,,,,,,
1,2016-01-01,-954.830591,NaN,NaN,NaN,NaN
2,2016-02-01,-954.830591,NaN,NaN,NaN,NaN
3,2016-03-01,-954.830591,NaN,NaN,NaN,NaN
4,2016-04-01,-954.830591,NaN,NaN,NaN,NaN
5,2016-05-01,-954.830591,NaN,NaN,NaN,NaN


In [32]:
df['Principal'] = npf.ppmt(interest_rate/payments_year, df.index, years*payments_year, principal)
df['Interest'] = npf.ipmt(interest_rate/payments_year, df.index, years*payments_year, principal)

In [33]:
df.head()

,Payment_Date,Payment,Principal,Interest,Addl_principal,Balance
Period,,,,,,
1,2016-01-01,-954.830591,-288.163924,-666.666667,NaN,NaN
2,2016-02-01,-954.830591,-289.124471,-665.706120,NaN,NaN
3,2016-03-01,-954.830591,-290.088219,-664.742372,NaN,NaN
4,2016-04-01,-954.830591,-291.055180,-663.775411,NaN,NaN
5,2016-05-01,-954.830591,-292.025364,-662.805227,NaN,NaN


In [35]:
def amortize(principal, interest_rate, years, addl_principal=0, annual_payments=12, start_date=date.today()):
    pmt = -round(npf.pmt(interest_rate/annual_payments, years*annual_payments, principal), 2)
    # initialize the variables to keep track of the periods and running balances
    p = 1
    beg_balance = principal
    end_balance = principal
    
    while end_balance > 0:
        
        # Recalculate the interest based on the current balance
        interest = round(((interest_rate/annual_payments) * beg_balance), 2)
        
        # Determine payment based on whether or not this period will pay off the laon
        pmt = min(pmt, beg_balance + interest)
        principal = pmt - interest
        
        # Ensure additional payment gets adjusted if the loan is beign paid off
        addl_principal = min(addl_principal, beg_balance - principal)
        end_balance = beg_balance - (principal + addl_principal)
        
        yield OrderedDict([('Month', start_date),
                           ('Period', p),
                           ('Begin Balance', beg_balance),
                           ('Payment', pmt),
                           ('Principal', principal),
                           ('Interest', interest),
                           ('Additional_Payment', addl_principal),
                           ('End Balance', end_balance)])
        
        p += 1
        start_date += relativedelta(months=1)
        beg_balance = end_balance

In [36]:
schedule = pd.DataFrame(amortize(700000, .04, 30, addl_principal=200, start_date=(date(2016, 1, 1))))

,Month,Period,Begin Balance,Payment,Principal,Interest,Additional_Payment,End Balance
0,2016-01-01,1,700000.00,3341.91,1008.58,2333.33,200.0,698791.42
1,2016-02-01,2,698791.42,3341.91,1012.61,2329.30,200.0,697578.81
2,2016-03-01,3,697578.81,3341.91,1016.65,2325.26,200.0,696362.16
3,2016-04-01,4,696362.16,3341.91,1020.70,2321.21,200.0,695141.46
4,2016-05-01,5,695141.46,3341.91,1024.77,2317.14,200.0,693916.69
...,...,...,...,...,...,...,...,...
319,2042-08-01,320,14413.65,3341.91,3293.86,48.05,200.0,10919.79
320,2042-09-01,321,10919.79,3341.91,3305.51,36.40,200.0,7414.28
321,2042-10-01,322,7414.28,3341.91,3317.20,24.71,200.0,3897.08
322,2042-11-01,323,3897.08,3341.91,3328.92,12.99,200.0,368.16
